In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
def weighted_random_recommendation(items_weights, n=5):
      
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = items_weights['item_id'] # список items
    weights = items_weights['item_weight']# список весов items
    
    recs = np.random.choice(items, n, p = weights, replace=False) # выбираем item, n-ое кол-во, с преимуществом веса по p
    
    
    return recs.tolist()

In [6]:
%%time
# your_code
# Подготовка данных
# вес каждого товара
item_weight = data_train.groupby(by=['item_id'], as_index=True).sum(['sales_value']) # находим сумму по каждому товару
item_weight['item_id'] = item_weight.index
item_weight['item_weight'] =item_weight['sales_value']/ item_weight['sales_value'].sum() # вычисляем вес каждой товарной позиции
name_cols = item_weight.columns.values # создаем столбец по индексу агрегата
name_cols = np.delete(name_cols, np.argwhere(name_cols == ['item_weight']))
name_cols = np.delete(name_cols, np.argwhere(name_cols == ['item_id']))
item_weight = item_weight.drop(columns = name_cols, axis = 1) # убираем ненужные столбцы

popular_recs = weighted_random_recommendation(item_weight, n=5)
popular_recs

Wall time: 620 ms


[835017, 1066506, 916122, 1109370, 866548]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [8]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [16]:
# your_code

def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = list(bought_list)
    recommended_list = list(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

for col_name in result.columns[2:]:
    print(f"{col_name}: {result.apply(lambda x: precision_at_k(x[col_name], x['actual']), axis=1).mean()}")
print("Лучший: cosine")

random_recommendation: 25.090401567091085
popular_recommendation: 23.425857002938294
itemitem: 27.002742409402547
cosine: 27.864152791381002
tfidf: 27.792066601371204
own_purchases: 8.23741429970617
Лучший: cosine


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [9]:
# your_code